In [ ]:
import pygame
import math
import random
import time

pygame.init()

# FOR TUNING
PREDICTION_HORIZON = 60
SAFE_CHECK_HORIZON = PREDICTION_HORIZON // 2
VIS_PRED_STEPS = 40

# GRID
GRID_SPACING = 40
GRID_MARGIN = 0
GRID_EVAL_EVERY_N = 6
RECOMPUTE_MOVE_THRESH = 2.0
w, h = 800, 600
BASIN_ALPHA = 120

# SPEEDS
FPS = 120
evader_speed = 6.0
pursuer_speed = 3.0
rel_speed = evader_speed / pursuer_speed
if rel_speed <=3.0:
    alpha_avoidance = 0.99
else:
    alpha_avoidance = 1
tangent_offset = math.pi / 6 #tangent evasion, nominal contorller
print(tangent_offset)

#GOALS
evader_radius, pursuer_radius, goal_radius = 15, 15, 20
max_goals = 10
MIN_GOAL_DIST_FROM_PURSUER = 100  # Minimum distance goal must be from pursuer

# CANDIDATES
NUM_CANDIDATES = 12
SAFE_MARGIN = 2.0
BUFFER_DIST = 10  # Early-warning safety buffer
unsafe_frames = 0

# COLORS
WHITE = (255,255,255)
BLUE = (50,150,255)
RED = (255,80,80)
GREEN = (50,200,50)
BLACK = (0,0,0)
PRED_E_COLOR = (0,120,255)
PRED_P_COLOR = (220,30,30)
BASIN_COLOR = (255,80,80, BASIN_ALPHA)

# initial state
evader_x = w//4
evader_y = h//2
pursuer_x = 3*w//4
pursuer_y = h//2
goal_counter = 0
goal_times = []

# SCREEN
screen = pygame.display.set_mode((w, h))
pygame.display.set_caption("Fixed Switching Filter with Buffer & Tangent Escape")
clock = pygame.time.Clock()

#SPAWN GOALS STRATEGICALLY
def spawn_goal_away_from_pursuer(pux, puy):
    max_attempts = 100
    for _ in range(max_attempts):
        gx = random.randint(0, w)
        gy = random.randint(0, h)
        dist = math.hypot(gx - pux, gy - puy)
        if dist >= MIN_GOAL_DIST_FROM_PURSUER:
            return gx, gy
    # Fallback: if we can't find a good spot after many attempts, return a position anyway
    return random.randint(0, w), random.randint(0, h)

# Spawn the initial goal
goal_x, goal_y = spawn_goal_away_from_pursuer(pursuer_x, pursuer_y)
goal_spawn_time = pygame.time.get_ticks() / 1000.0

#Star tthe game
running = True
game_over = False
frame_count = 0
_last_pursuer_pos = (pursuer_x, pursuer_y)
choice_idx = 0


#  IMMEDIATE EARLY WARNING
def early_warning(evx, evy, pux, puy):
    dx = evx - pux
    dy = evy - puy
    dist = math.hypot(dx, dy)
    return dist < (evader_radius + pursuer_radius + BUFFER_DIST)

########################################################
# PREDICTED PATHS
########################################################
def get_predicted_paths(evx, evy, pux, puy, mode='SAFETY', steps=VIS_PRED_STEPS):  # get the predicted paths
    e_x, e_y = float(evx), float(evy) # evader position
    p_x, p_y = float(pux), float(puy) # pursuer position
    e_path, p_path = [], [] # empty lists to store the paths
    for _ in range(steps): # iterate over the number of steps
        dx = e_x - p_x
        dy = e_y - p_y
        dist = math.hypot(dx, dy)
        if mode == 'GOAL': # if the mode is goal
            gdx = goal_x - e_x # distance between goal and evader
            gdy = goal_y - e_y
            gdist = math.hypot(gdx, gdy)
            if gdist > 1e-6: # if the distance is greater than 0, the evader is moving
                ev_vx = (gdx / gdist) * evader_speed
                ev_vy = (gdy / gdist) * evader_speed
            else:
                ev_vx, ev_vy = 0.0, 0.0
        else: # if the mode is safety
            if dist > 1e-6: # if the distance is greater than 0, the evader is moving
                ev_vx = (dx / dist) * evader_speed
                ev_vy = (dy / dist) * evader_speed
            else:
                ev_vx, ev_vy = 0.0, 0.0
        e_x += ev_vx # update evader position
        e_y += ev_vy
        lookahead = 3.0
        pred_ex = e_x + ev_vx * lookahead # predicted evader position
        pred_ey = e_y + ev_vy * lookahead
        pdx = pred_ex - p_x # distance between predicted evader and pursuer
        pdy = pred_ey - p_y
        pdist = math.hypot(pdx, pdy) # distance between predicted evader and pursuer
        if pdist > 1e-6: # if the distance is greater than 0, the pursuer is moving
            p_x += (pdx / pdist) * pursuer_speed
            p_y += (pdy / pdist) * pursuer_speed
        e_x = max(evader_radius, min(w - evader_radius, e_x))
        e_y = max(evader_radius, min(h - evader_radius, e_y))
        p_x = max(pursuer_radius, min(w - pursuer_radius, p_x))
        p_y = max(pursuer_radius, min(h - pursuer_radius, p_y))
        e_path.append((e_x, e_y))
        p_path.append((p_x, p_y))
        if math.hypot(e_x - p_x, e_y - p_y) <= evader_radius + pursuer_radius:
            break
    return e_path, p_path   #return the paths of both agents
    
########################################################
# SHORT_TERM CAPTURE SIM DETAILS
########################################################
def will_be_captured(evx, evy, pux, puy, horizon=PREDICTION_HORIZON):
    e_x, e_y = float(evx), float(evy) # evader position
    p_x, p_y = float(pux), float(puy) # pursuer position
    for _ in range(horizon):
        dx = e_x - p_x # distance between evader and pursuer
        dy = e_y - p_y
        dist = math.hypot(dx, dy) # distance between evader and pursuer
        if dist <= (evader_radius + pursuer_radius): # if the distance is less than the sum of the radii, the evader is captured
            return True
        if dist > 1e-6: # if the distance is greater than 0, the evader is moving
            ev_vx = (dx / dist) * evader_speed # evader velocity    
            ev_vy = (dy / dist) * evader_speed
        else:
            ev_vx, ev_vy = 0.0, 0.0
        e_x += ev_vx # update evader position
        e_y += ev_vy
        e_x = max(evader_radius, min(w - evader_radius, e_x)) # keep evader in bounds
        e_y = max(evader_radius, min(h - evader_radius, e_y))
        lookahead = 3.0
        pred_ex = e_x + ev_vx * lookahead # predicted evader position
        pred_ey = e_y + ev_vy * lookahead
        pdx = pred_ex - p_x # distance between predicted evader and pursuer
        pdy = pred_ey - p_y
        pdist = math.hypot(pdx, pdy) # distance between predicted evader and pursuer
        if pdist > 1e-6: # if the distance is greater than 0, the pursuer is moving
            p_x += (pdx / pdist) * pursuer_speed
        p_x = max(pursuer_radius, min(w - pursuer_radius, p_x))
        p_y = max(pursuer_radius, min(h - pursuer_radius, p_y))
        if math.hypot(e_x - p_x, e_y - p_y) <= evader_radius + pursuer_radius:
            return True
    return False

########################################################
#  LONG_TERM BASIN CAPTURE
########################################################
grid_xs = list(range(GRID_MARGIN + GRID_SPACING//2, w, GRID_SPACING)) # grid x coordinates
grid_ys = list(range(GRID_MARGIN + GRID_SPACING//2, h, GRID_SPACING)) # grid y coordinates
nx = len(grid_xs) # number of grid cells in x direction
ny = len(grid_ys) # number of grid cells in y direction
basin_grid = [[False]*ny for _ in range(nx)] # grid of boolean values indicating if the cell is in the basin
basin_stamp = -1 # frame number when the basin was last computed

def recompute_basin(p_x, p_y): # recompute the basin
    global basin_grid, basin_stamp # global variables
    for i, gx in enumerate(grid_xs): # iterate over all grid cells
        for j, gy in enumerate(grid_ys):
            basin_grid[i][j] = will_be_captured(gx, gy, p_x, p_y, horizon=SAFE_CHECK_HORIZON)
    basin_stamp = frame_count

def is_in_sampled_basin(x, y): # check if the cell is in the basin
    i = int(round((x - (GRID_SPACING/2)) / GRID_SPACING)) # index of the grid cell
    j = int(round((y - (GRID_SPACING/2)) / GRID_SPACING)) # index of the grid cell
    i = max(0, min(nx-1, i)) # keep the index in bounds
    j = max(0, min(ny-1, j)) # keep the index in bounds
    return basin_grid[i][j] # return the value of the grid cell

########################################################
# now that we have the immediate, short term, long term options, we can choose the evader action
# Choose EVADER ACTION
########################################################
def choose_evader_action(evx, evy, goalx, goaly, pux, puy):
    gdx = goalx - evx
    gdy = goaly - evy # distance between goal and evader
    gdist = math.hypot(gdx, gdy) # distance between goal and evader
    goal_dir = math.atan2(gdy, gdx) if gdist > 1e-6 else 0.0 # goal direction

    pdx = evx - pux # distance between pursuer and evader
    pdy = evy - puy
    pdist = math.hypot(pdx, pdy) # distance between pursuer and evader
    away_dir = math.atan2(pdy, pdx) if pdist > 1e-6 else goal_dir # away direction

    # tangent escape directions
    angle_to_pursuer = math.atan2(evader_y - pursuer_y, evader_x - pursuer_x)
    tangent_angles = [angle_to_pursuer + tangent_offset, angle_to_pursuer - tangent_offset]

    candidates = [goal_dir, away_dir] + tangent_angles # candidates for the evader action
    for k in range(NUM_CANDIDATES):
        frac = k / max(1, NUM_CANDIDATES - 1)
        diff = ((away_dir - goal_dir + math.pi) % (2*math.pi)) - math.pi
        candidates.append(goal_dir + diff * frac)

    safe_candidates = [] # safe candidates
    scored_candidates = [] # scored candidates (if unsafe, what to do next)

    for idx, angle in enumerate(candidates): # iterate over the candidates
        nx = evx + math.cos(angle) * evader_speed
        ny = evy + math.sin(angle) * evader_speed
        nx = max(evader_radius, min(w - evader_radius, nx))
        ny = max(evader_radius, min(h - evader_radius, ny))

        #be careful around edges
        # Edge-awareness: nudge angles away from walls if candidate is too close
        if nx < evader_radius*2:
            angle += math.radians(30)  # left wall
        elif nx > w - evader_radius*2:
            angle -= math.radians(30)  # right wall
        if ny < evader_radius*2:
            angle += math.radians(30)  # top wall
        elif ny > h - evader_radius*2:
            angle -= math.radians(30)  # bottom wall

        # recompute candidate position after nudge
        nx = evx + math.cos(angle) * evader_speed # new candidate position
        ny = evy + math.sin(angle) * evader_speed
        nx = max(evader_radius, min(w - evader_radius, nx))
        ny = max(evader_radius, min(h - evader_radius, ny))


        # reject moves that reduce distance to pursuer
        if math.hypot(nx - pux, ny - puy) < alpha_avoidance * math.hypot(evx - pux, evy - puy): # if the new candidate position is too close to the pursuer
            continue

        if not is_in_sampled_basin(nx, ny): # if the new candidate position is not in the basin
            ang_diff = abs(((angle - goal_dir + math.pi) % (2*math.pi)) - math.pi) # angle difference
            safe_candidates.append((ang_diff, idx, nx, ny, angle)) # add the candidate to the safe candidates
        else:
            captured = will_be_captured(nx, ny, pux, puy, horizon=SAFE_CHECK_HORIZON) # if the new candidate position is in the basin   
            if captured: # if the new candidate position is captured
                score = 0.0 
            else: # if the new candidate position is not captured
                score = 0.5 
            scored_candidates.append((score, idx, nx, ny, angle)) # add the candidate to the scored candidates

    if safe_candidates: # if there are safe candidates
        safe_candidates.sort(key=lambda t: (t[0], t[1])) # sort the safe candidates
        _, chosen_idx, nx, ny, _ = safe_candidates[0] # choose the first safe candidate
        return nx, ny, chosen_idx, True # return the chosen candidate
    if scored_candidates: # if there are scored candidates
        scored_candidates.sort(key=lambda t: (-t[0], t[1])) # sort the scored candidates
        _, chosen_idx, nx, ny, _ = scored_candidates[0] # choose the first scored candidate
        return nx, ny, chosen_idx, False # return the chosen candidate
    return evx, evy, 0, False






# main loop
while running:

    #start the game
    clock.tick(FPS)
    screen.fill(WHITE)
    frame_count += 1

    # handle quit event
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            game_over = True

    #otherwise if not quit:
    if not game_over:
        dxp = pursuer_x - _last_pursuer_pos[0]
        dyp = pursuer_y - _last_pursuer_pos[1]
        if frame_count == 1 or math.hypot(dxp, dyp) >= RECOMPUTE_MOVE_THRESH or frame_count % GRID_EVAL_EVERY_N == 0: #if the pursuer has moved enough or the grid needs to be recomputed
            recompute_basin(pursuer_x, pursuer_y) # recompute the basin
            _last_pursuer_pos = (pursuer_x, pursuer_y) # update the last pursuer position

        # choose the evader action
        next_ex, next_ey, choice_idx, step_safe = choose_evader_action(
            evader_x, evader_y, goal_x, goal_y, pursuer_x, pursuer_y
        )

        # check if the evader is in the basin or if the early warning is triggered
        current_unsafe = is_in_sampled_basin(evader_x, evader_y) or early_warning(evader_x, evader_y, pursuer_x, pursuer_y)
        unsafe_frames = unsafe_frames + 1 if current_unsafe else max(unsafe_frames - 1, 0)
        if current_unsafe:
            mode = "SAFETY"
        else:
            mode = "GOAL"

        # update the evader position
        evader_x, evader_y = next_ex, next_ey

        # pursuer pure pursuit calc
        pdx = evader_x - pursuer_x # distance between evader and pursuer
        pdy = evader_y - pursuer_y
        pdist = math.hypot(pdx, pdy) # distance between evader and pursuer
        if pdist > 1e-6: # if the distance is greater than 0, the pursuer is moving
            pursuer_x += (pdx / pdist) * pursuer_speed
            pursuer_y += (pdy / pdist) * pursuer_speed
        pursuer_x = max(pursuer_radius, min(w - pursuer_radius, pursuer_x)) # keep pursuer in bounds
        pursuer_y = max(pursuer_radius, min(h - pursuer_radius, pursuer_y))
        evader_x = max(evader_radius, min(w - evader_radius, evader_x)) # keep evader in bounds
        evader_y = max(evader_radius, min(h - evader_radius, evader_y))

        # collisions / goals
        if math.hypot(evader_x - pursuer_x, evader_y - pursuer_y) <= evader_radius + pursuer_radius:
            game_over = True # game over
            running = False # stop the game
        if goal_counter < max_goals and math.hypot(evader_x - goal_x, evader_y - goal_y) <= evader_radius + goal_radius: # if the evader is close to the goal
            goal_times.append(pygame.time.get_ticks() / 1000.0 - goal_spawn_time) # add the time it took to reach the goal
            goal_counter += 1 # increment the goal counter
            if goal_counter < max_goals: # if the goal counter is less than the max goals   
                goal_x, goal_y = spawn_goal_away_from_pursuer(pursuer_x, pursuer_y) # spawn a new goal
                goal_spawn_time = pygame.time.get_ticks() / 1000.0 # update the goal spawn time 
            else:
                game_over = True # game over
                running = False

    ### #################################
    ########## everything below this is VISUALIZATION ##########
    ####################################
    pred_e_path, pred_p_path = get_predicted_paths(evader_x, evader_y, pursuer_x, pursuer_y, mode=mode)

    # overlay for the basin
    overlay = pygame.Surface((w, h), flags=pygame.SRCALPHA)
    overlay.fill((0,0,0,0))
    for i, gx in enumerate(grid_xs):
        for j, gy in enumerate(grid_ys):
            if basin_grid[i][j]:
                left = gx - GRID_SPACING//2
                top  = gy - GRID_SPACING//2
                rect = pygame.Rect(left, top, GRID_SPACING, GRID_SPACING)
                overlay.fill((BASIN_COLOR[0], BASIN_COLOR[1], BASIN_COLOR[2], BASIN_ALPHA), rect)
    screen.blit(overlay, (0,0))

    # draw the predicted paths
    for i in range(1, len(pred_e_path)):
        pygame.draw.line(screen, PRED_E_COLOR,
                         (int(pred_e_path[i-1][0]), int(pred_e_path[i-1][1])),
                         (int(pred_e_path[i][0]), int(pred_e_path[i][1])), 2)
    for i in range(1, len(pred_p_path)):
        pygame.draw.line(screen, PRED_P_COLOR,
                         (int(pred_p_path[i-1][0]), int(pred_p_path[i-1][1])),
                         (int(pred_p_path[i][0]), int(pred_p_path[i][1])), 2)

    # draw the pursuer, evader, and goal
    pygame.draw.circle(screen, RED, (int(pursuer_x), int(pursuer_y)), pursuer_radius)
    pygame.draw.circle(screen, BLUE, (int(evader_x), int(evader_y)), evader_radius)
    pygame.draw.circle(screen, GREEN, (int(goal_x), int(goal_y)), goal_radius)
    pygame.draw.line(screen, BLACK, (int(evader_x), int(evader_y)), (int(pursuer_x), int(pursuer_y)), 1)

    # draw the text
    font = pygame.font.Font(None, 24)
    if not game_over:
        screen.blit(font.render(f"Goals: {goal_counter}/{max_goals}", True, GREEN), (10, 8))
        screen.blit(font.render(f"Mode: {mode}", True, RED if mode!="GOAL" else GREEN), (10, 32))
        screen.blit(font.render(f"Grid: {nx}x{ny}", True, BLACK), (10, 56))
        screen.blit(font.render(f"Choice idx: {choice_idx}", True, BLACK), (10, 80))
        screen.blit(font.render(f"Pred H: {PREDICTION_HORIZON}", True, BLACK), (10, 104))
    else:
        if goal_counter >= max_goals:
            msg = font.render(f"VICTORY! {goal_counter} goals", True, GREEN)
        else:
            msg = font.render("CAUGHT - game over", True, RED)
        screen.blit(msg, msg.get_rect(center=(w//2, h//2)))

    # update the screen
    pygame.display.flip()

pygame.quit()


0.5235987755982988
